In [ ]:
# mqtt control of wheeled card
# there is another repo called wheeliephone that this copies back from
# Small ESP8266

#      RST          |  GPIO1   TX
#       A0          |  GPIO3   RX
#       D0  GPIO16  |  GPIO5   D1  SCL
# SCK   D5  GPIO14  |  GPIO4   D2  SDA
# MISO  D6  GPIO12  |  GPIO0   D3
# MOSI  D7  GPIO13  |  GPIO2   D4  LED
# SS    D8  GPIO15  |          GND
#      3V3          |          5V

In [6]:
%serialconnect

serial exception on close write failed: [Errno 5] Input/output error
Connecting to --port=/dev/ttyUSB0 --baud=115200 
Ready.


In [19]:
%sendtofile --source wifimqttconnections.txt config.txt
%sendtofile --append config.txt

pinled          -2
mAen            15
mAfore          0
mAback          16
mBen            13
mBfore          14
mBback          12

Sent 7 lines (131 bytes) to config.txt.


In [7]:
%sendtofile --source stdmqttas.py
%sendtofile --quiet --source utils.py

Sent 81 lines (2677 bytes) to stdmqttas.py.
Sent 32 lines (969 bytes) to utils.py.


In [4]:
%sendtofile main.py

import os, time, sys, machine, itertools, time
from machine import Pin, PWM
from stdmqttas import fconfig, config, mqttconnecttask, callbackcmdtask
import uasyncio as asyncio
from stdmqttas import pinled, flashpinled, shortmac, topicstem, itertools_count
from mqtt_as import MQTTClient

import network
network.WLAN().active(0)  # disable the connection at startup
#flashpinled(5, 300, 300)

from machine import Pin, PWM
mApwm = PWM(Pin(int(fconfig["mAen"])))
mAfore = Pin(int(fconfig["mAfore"]), Pin.OUT)
mAback = Pin(int(fconfig["mAback"]), Pin.OUT)
mApwm.freq(10)
mApwm.duty(0)

mBpwm = PWM(Pin(int(fconfig["mBen"])))
mBfore = Pin(int(fconfig["mBfore"]), Pin.OUT)
mBback = Pin(int(fconfig["mBback"]), Pin.OUT)
mBpwm.freq(10)
mBpwm.duty(0)

timeZeroBy = 1
async def zeroingtask():
    global timeZeroBy
    while True:
        if timeZeroBy != 0:
            if time.ticks_ms() > timeZeroBy:
                timeZeroBy = 0
                mAfore.value(0)
                mAback.value(0)
                mBfore.value(0)
                mBback.value(0)
                pinled.value(1)
        await asyncio.sleep_ms(50)


def updatewheels(msg):
    global timeZeroBy
    try:
        s = msg.split()
        timeZeroBy = time.ticks_ms() + int(s[6])
        mApwm.duty(int(s[0]))
        mAfore.value(int(s[1]))
        mAback.value(int(s[2]))
        mBpwm.duty(int(s[3]))
        mBfore.value(int(s[4]))
        mBback.value(int(s[5]))
        pinled.value(0)
    except Exception as e:
        print("Bad wheels", msg, e)

topicboardwheels = topicstem+"/wheels"
topiccmd = (topicstem+"/cmd").encode()
topicstatus = topicstem+"/status"
topicreply = topicstem+"/reply"
topiconline = topicstem+"/online"
topicip = topicstem+"/ip"

for l in ["500 1 0 500 1 0 1", "500 0 1 500 0 1 1", 
          "500 1 0 500 0 1 1", "500 0 1 500 1 0 1", "500 0 0 500 0 0 1"]:
    updatewheels(l)
    time.sleep_ms(500)

def callbackcmd(topic, msg, retained):
    print(topic, msg)
    if topic == topiccmd:
        aloop.create_task(callbackcmdtask(client, topicreply, msg))
    else:
        updatewheels(msg)
    
async def onconnecttask(client):
    print("subscribing to", topicboardwheels)
    ipnumber = client._sta_if.ifconfig()[0]
    print("ipnumber", ipnumber)
    await client.publish(topicstatus, ipnumber, retain=True)
    await client.subscribe(topiccmd, 1)
    await client.subscribe(topicboardwheels, 1)
    
config['subs_cb'] = callbackcmd
config['connect_coro'] = onconnecttask
config['will'] = (topicstatus, "offline", True)
client = MQTTClient(config)
client.DEBUG = True
  
aloop = asyncio.get_event_loop()
aloop.create_task(zeroingtask())
aloop.create_task(mqttconnecttask(client))
aloop.run_forever()

#left = 800 1 0 800 1 0 50
#right = 800 0 1 800 0 1 50
#fore = 1023 0 1 1023 1 0 200
#back = 800 1 0 800 0 1 200
# mosquitto_pub -h mosquitto.doesliverpool.xyz -t esp8266/4A3701/wheels -m "800 1 0 800 1 0 50"

Sent 97 lines (2913 bytes) to main.py.
